In [1]:
print("ok")

ok


In [1]:
%pwd

'd:\\My projects\\Doctor_bot\\research'

In [2]:
import os
os.chdir("../")

In [3]:
%pwd

'd:\\My projects\\Doctor_bot'

In [4]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [5]:
# Extract Data from the PDF file
def load_pdf_file(data):
    loader = DirectoryLoader(data,
                             glob="*.pdf",
                             loader_cls=PyPDFLoader)
    documents=loader.load()

    return documents

In [6]:
extracted_data=load_pdf_file(data='Data/')

In [7]:
# extracted_data

In [8]:
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)
    text_chunk=text_splitter.split_documents(extracted_data)
    return text_chunk

In [9]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks",len(text_chunks))

Length of Text Chunks 5859


In [10]:
# text_chunks

In [11]:
from langchain.embeddings import HuggingFaceEmbeddings

In [12]:
from langchain_community.embeddings import HuggingFaceEmbeddings

def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(
        model_name='sentence-transformers/all-MiniLM-L6-v2'
    )
    return embeddings


In [13]:
embedding = download_hugging_face_embeddings()

C:\Users\gjyot\AppData\Local\Temp\ipykernel_6704\119269335.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
d:\Users\gjyot\anaconda3\envs\DCbot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
query_result = embedding.embed_query("hello Saurabh")
print("Length", len(query_result))

Length 384


In [15]:
# query_result

In [16]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [33]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
OPENROUTER_API_KEY=os.environ.get('OPENROUTER_API_KEY')
api_base = os.getenv("OPENROUTER_API_BASE")

In [18]:
%pwd

'd:\\My projects\\Doctor_bot'

In [ ]:
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENROUTER_API_KEY"] = OPENROUTER_API_KEY 


In [20]:
from pinecone import Pinecone

pinecone_api_key = PINECONE_API_KEY

pc = Pinecone(api_key=pinecone_api_key)

In [21]:
from pinecone import ServerlessSpec

index_name = "medical-bot"  # change if desired

if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )

index = pc.Index(index_name)

In [22]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embedding, 
)

In [23]:
# load Existing index
from langchain_pinecone import PineconeVectorStore
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embedding
)

In [24]:
docsearch

In [25]:
retriever = docsearch.as_retriever(search_type="similarity",search_kwargs={"k":3})

In [26]:
retriever_docs = retriever.invoke("What is Abdominal ultrasound")


In [27]:
retriever_docs

[Document(id='f566aba0-b984-44c1-b1ef-1cce54616588', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 18.0, 'page_label': '19', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Medical_book.pdf', 'total_pages': 637.0}, page_content='an abdominal ultrasound, as a fetus in the early stages of\ndevelopment could be injured by ultrasound meant to\nprobe deeply recessed abdominal organs.\nNormal results\nAs a diagnostic imaging technique, a normal abdom-\ninal ultrasound is one that indicates the absence of the\nsuspected condition that prompted the scan. For example,\nsymptoms such as a persistent cough, labored breathing,\nand upper abdominal pain suggest the possibility of,\namong other things, an abdominal aortic aneurysm. An'),
 Document(id='98208934-d2c5-4033-b2bf-26f45f2a6a7a', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 18.0

In [28]:
# import os
# from dotenv import load_dotenv
# from langchain_openai import ChatOpenAI

# # Load environment variables
# load_dotenv()

# # Get API values
# api_key = os.getenv("OPENROUTER_API_KEY")
# api_base = os.getenv("OPENROUTER_API_BASE")

# # Initialize the model
# llm = ChatOpenAI(
#     openai_api_key=api_key,
#     openai_api_base=api_base,
#     model="openai/gpt-4o-mini",  # pick any model from OpenRouter
# )

# # Test
# response = llm.invoke("Hello, OpenRouter with LangChain + .env!")
# print(response)


In [41]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(
    openai_api_key=OPENROUTER_API_KEY,
    openai_api_base=api_base,
    model="gpt-4o-mini"
)

In [43]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


system_prompt = (
    "You are an assistant for qustion-answering task."
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [44]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [ ]:
# def format_rag_output(rag_result):
#     # Extract the answer
#     answer = rag_result.get("answer", "No answer found.")
    
#     # Extract unique sources
#     sources = []
#     for doc in rag_result.get("context", []):
#         page = doc.metadata.get("page_label") or doc.metadata.get("page")
#         source = doc.metadata.get("source")
#         if source and page:
#             sources.append(f"{source} (page {page})")
#         elif source:
#             sources.append(source)
    
#     # Deduplicate sources
#     sources = list(set(sources))
    
#     # Build final formatted output
#     output = f"📖 **Answer:**\n{answer}\n\n"
#     if sources:
#         output += "🔎 **Sources:**\n" + "\n".join([f"- {s}" for s in sources])
    
#     return output


In [45]:
response = rag_chain.invoke({"input": "What is Abortion, therapeutic"})
# print(format_rag_output(response))
print(response)

{'input': 'What is Abortion, therapeutic', 'context': [Document(id='ee9031af-5c9e-446f-b636-28dfb847f1ed', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 22.0, 'page_label': '23', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Medical_book.pdf', 'total_pages': 637.0}, page_content='ORGANIZATIONS\nThe American Society for Reproductive Medicine. 1209 Mont-\ngomery Highway, Birmingham, AL 35216-2809. (205)\n978-5000. <http://www.asrm.org>.\nThe Alan Guttmacher Institute. 120 Wall Street, New York, NY\n10005. (212) 248-1111. <http://www.agi-usa.org>.\nMeghan M. Gourley\nAbortion, spontaneous see Miscarriage\nAbortion, therapeutic\nDefinition\nTherapeutic abortion is the intentional termination\nof a pregnancy before the fetus can live independently.'), Document(id='b6f5f3e7-d767-49c8-b4e0-38ec33384bdf', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15